## Day 23: Experimental Emergency Teleportation

https://adventofcode.com/2018/day/23

### Part 1

In [1]:
from parse import parse


def parse_nanobots(data):
    nanobots = {}
    
    for line in data:
        x, y, z, r = parse('pos=<{:d},{:d},{:d}>, r={:d}', line)
        nanobots[(x, y, z)] = r
        
    return nanobots


def manhattan_distance(p1, p2):
    return sum(abs(p - q) for p, q in zip(p1, p2))


def number_in_range_of_most_powerful(nanobots):
    most_powerful = max(nanobots, key=lambda k: nanobots[k])
    return sum(1 for nb in nanobots
               if manhattan_distance(most_powerful, nb) <= nanobots[most_powerful])


test_data = '''pos=<0,0,0>, r=4
pos=<1,0,0>, r=1
pos=<4,0,0>, r=3
pos=<0,2,0>, r=1
pos=<0,5,0>, r=3
pos=<0,0,3>, r=1
pos=<1,1,1>, r=1
pos=<1,1,2>, r=1
pos=<1,3,1>, r=1'''.splitlines()

test_nanobots = parse_nanobots(test_data)
assert number_in_range_of_most_powerful(test_nanobots) == 7

In [2]:
nanobots = parse_nanobots(open('input', 'r'))
number_in_range_of_most_powerful(nanobots)

889

That's straightforward.

### Part 2

Hmm, this isn't, need to have a bit of a think.

#### Some time later

OK, this is tricky. The number of possible coordinates is enormous, far too large for any brute force approach, but I can't think of an analytical approach. We need an exact solution, so I'll try a directed brute force. Whether this will run in an acceptable time without running out of memory will depend on the distribution of maxima, if they're concentrated in one place then it should be OK but if they're regularly distributed through space then it could get stuck.

Start with a cube large enough to include every nanobot. Iteratively select the cube in range of the most nanobots and split into eight smaller cubes. Continue until the cube in range of the most nanobots only has one coordinate. 

As the width of the cube is halved at each stage the initial cube needs to have a width that is a power of two.

In [3]:
import math

def initial_cube(nanobots):
    xyzs = list(zip(*nanobots))
    ranges = [max(xs) - min(xs) for xs in xyzs]
    mins = [min(xs) for xs in xyzs]
    corner = tuple(mn for mn in mins)
    actual_width = max(ranges)
    width = 2 ** math.ceil(math.log(actual_width, 2))
    return (corner, width)

In [4]:
test_cube_1 = initial_cube(test_nanobots)
test_cube_1

((0, 0, 0), 8)

In [5]:
def split_cube(cube):
    (x, y, z), width = cube
    result = []

    new_width = width // 2
    for dx in (0, new_width):
        for dy in (0, new_width):
            for dz in (0, new_width):
                result.append(((x + dx, y + dy, z + dz), new_width))
                
    return result

test_cubes_2 = split_cube(test_cube_1)
test_cubes_2

[((0, 0, 0), 4),
 ((0, 0, 4), 4),
 ((0, 4, 0), 4),
 ((0, 4, 4), 4),
 ((4, 0, 0), 4),
 ((4, 0, 4), 4),
 ((4, 4, 0), 4),
 ((4, 4, 4), 4)]

In [6]:
split_cube(test_cubes_2[2])

[((0, 4, 0), 2),
 ((0, 4, 2), 2),
 ((0, 6, 0), 2),
 ((0, 6, 2), 2),
 ((2, 4, 0), 2),
 ((2, 4, 2), 2),
 ((2, 6, 0), 2),
 ((2, 6, 2), 2)]

Use a priority queue of cubes, ordered by the number of nanobots in range to ensure we get the correct solution, then width so we don't waste time on larger cubes when there's a possible answer in the queue, then Manhattan distance from the origin to match the specification of the problem.

In [7]:
from heapq import heappop, heappush, heapify


ORIGIN = (0, 0, 0)


def nearest_in_range(x, min_x, max_x):
    if x < min_x:
        return min_x
    elif x > max_x:
        return max_x
    else:
        return x


def nanobots_in_range(cube, nanobots):
    corner, width = cube
    far_corner = [i + width - 1 for i in corner]
    number_in_range = 0
    
    for n in nanobots:
        nearest_point = [nearest_in_range(*xs) 
                         for xs in zip(n, corner, far_corner)]
        if manhattan_distance(nearest_point, n) <= nanobots[n]:
            number_in_range += 1
            
    return number_in_range
        

def best_coordinate(nanobots):
    start = initial_cube(nanobots)
    
    # Sorted queue by number of nanobots in range with
    # proximity to origin as a tie-break
    search = [(-nanobots_in_range(start, nanobots),
               start[1],
               manhattan_distance(start[0], ORIGIN),
               start)]
    
    while search:
        n, w, m, current_cube = heappop(search)
                    
        coordinates, width = current_cube
        
        if w == 1:
            return (m, coordinates, -n)
               
        for next_cube in split_cube(current_cube):
            bots_in_range = nanobots_in_range(next_cube, nanobots)
            if bots_in_range > 0:
                heappush(search, (-bots_in_range, next_cube[1],
                                  manhattan_distance(next_cube[0], ORIGIN),
                                  next_cube))
            
    return None

In [8]:
test_data_2 = '''pos=<10,12,12>, r=2
pos=<12,14,12>, r=2
pos=<16,12,12>, r=4
pos=<14,14,14>, r=6
pos=<50,50,50>, r=200
pos=<10,10,10>, r=5'''.splitlines()

test_nanobots_2 = parse_nanobots(test_data_2)
%time best_coordinate(test_nanobots_2)

CPU times: user 891 µs, sys: 138 µs, total: 1.03 ms
Wall time: 1.03 ms


(36, (12, 12, 12), 5)

That matches the test data.

In [9]:
%time best_coordinate(nanobots)

CPU times: user 1.17 s, sys: 2.44 ms, total: 1.17 s
Wall time: 1.18 s


(160646364, (56721513, 49483609, 54441242), 977)

That is considerably quicker than I was expecting, the problem data looks like it was constructed to be friendly to stupid approaches.